# Wine Quality

## Introduction:

*Loading Libraries*

In [3]:
library(tidyverse)
library(tidymodels)
library(gridExtra)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

*Reading in data*

In [5]:
data <- read_csv("data/WineQT.csv")
data

Parsed with column specification:
cols(
  `fixed acidity` = col_double(),
  `volatile acidity` = col_double(),
  `citric acid` = col_double(),
  `residual sugar` = col_double(),
  chlorides = col_double(),
  `free sulfur dioxide` = col_double(),
  `total sulfur dioxide` = col_double(),
  density = col_double(),
  pH = col_double(),
  sulphates = col_double(),
  alcohol = col_double(),
  quality = col_double(),
  Id = col_double()
)



fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
7.4,0.700,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,0
7.8,0.880,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,5,1
7.8,0.760,0.04,2.3,0.092,15,54,0.9970,3.26,0.65,9.8,5,2
11.2,0.280,0.56,1.9,0.075,17,60,0.9980,3.16,0.58,9.8,6,3
7.4,0.700,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,4
7.4,0.660,0.00,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,5,5
7.9,0.600,0.06,1.6,0.069,15,59,0.9964,3.30,0.46,9.4,5,6
7.3,0.650,0.00,1.2,0.065,15,21,0.9946,3.39,0.47,10.0,7,7
7.8,0.580,0.02,2.0,0.073,9,18,0.9968,3.36,0.57,9.5,7,8


*Cleaning and Wrangling*

[insert units here]

In [7]:
data <- mutate(data, quality = as_factor(quality))
data

fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
7.4,0.700,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,0
7.8,0.880,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,5,1
7.8,0.760,0.04,2.3,0.092,15,54,0.9970,3.26,0.65,9.8,5,2
11.2,0.280,0.56,1.9,0.075,17,60,0.9980,3.16,0.58,9.8,6,3
7.4,0.700,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,4
7.4,0.660,0.00,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,5,5
7.9,0.600,0.06,1.6,0.069,15,59,0.9964,3.30,0.46,9.4,5,6
7.3,0.650,0.00,1.2,0.065,15,21,0.9946,3.39,0.47,10.0,7,7
7.8,0.580,0.02,2.0,0.073,9,18,0.9968,3.36,0.57,9.5,7,8


In [5]:
clean_data <- rename(data, snoring_rate = sr, 
                     respiration_rate = rr, body_temperature = t, limb_movements = lm, 
                     blood_oxygen_levels = bo, eye_movements = rem, hours_sleep = sr_1, 
                     heart_rate = hr, stress_level = sl)

In [19]:
clean_data_split <- initial_split(clean_data, prop = 0.5, strata = stress_level)
clean_data_train <- training(clean_data_split)
clean_data_test <- testing(clean_data_split)

In [20]:
clean_data_train <- clean_data_train

snoring_rate,respiration_rate,body_temperature,limb_movements,blood_oxygen_levels,eye_movements,hours_sleep,heart_rate,stress_level
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
60.000,20.000,96.000,10.000,95.000,85.00,7.000,60.00,1
47.000,16.800,97.200,5.600,95.800,68.00,7.800,52.00,0
50.000,18.000,99.000,8.000,97.000,80.00,9.000,55.00,0
55.520,19.104,95.104,9.104,93.656,82.76,6.104,57.76,1
73.440,21.344,93.344,11.344,91.344,91.72,4.016,63.36,2
96.288,26.288,85.360,17.144,82.432,100.36,0.000,75.72,4
52.640,18.528,94.528,8.528,92.792,81.32,5.528,56.32,1
81.560,22.416,90.208,12.520,88.208,95.52,0.208,66.04,3
89.960,24.656,91.328,15.320,89.328,98.32,1.328,71.64,3
